In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import random
from google.colab import files

uploaded = files.upload()

X_train = np.load("x.npy")
y_train = np.array([1]*1000+[0]*1000, dtype="float32")

# 1. 資料增強函數
def augment_matrix(mat):

    mat = np.array(mat)


    # 隨機平移 (-1, 0, 1)
    def shift_matrix(mat, dx, dy):
        mat_shifted = np.zeros_like(mat)
        x_range = range(max(0, dx), min(8, 8+dx))
        y_range = range(max(0, dy), min(8, 8+dy))
        x_new_range = range(max(0, -dx), min(8, 8-dx))
        y_new_range = range(max(0, -dy), min(8, 8-dy))
        mat_shifted[np.ix_(y_new_range, x_new_range)] = mat[np.ix_(y_range, x_range)]
        return mat_shifted

    # 隨機旋轉 (0°, 90°, 180°, 270°)
    def rotate_matrix(mat, k):
        return np.rot90(mat, k)

    #
    def flip_matrix(mat, axis):
        return np.flip(mat, axis)

    augmented_mats = []

    # 原始矩陣
    augmented_mats.append(mat)

    # 平移增强
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx != 0 or dy != 0:
                augmented_mats.append(shift_matrix(mat, dx, dy))

    # 旋转增强
    for k in range(1, 4):  # 90°, 180°, 270°
        augmented_mats.append(rotate_matrix(mat, k))

    # 旋轉增強
    augmented_mats.append(flip_matrix(mat, 0))  # 水平翻转
    augmented_mats.append(flip_matrix(mat, 1))  # 垂直翻转

    # 添加噪音
    noisy_mat = mat.copy()
    noise_mask = np.random.random((8, 8)) < 0.1
    noisy_mat[noise_mask] = 1 - noisy_mat[noise_mask]
    augmented_mats.append(noisy_mat)

    return augmented_mats


def expand_dataset(X, y, target_size=10000):
    """将数据集扩展到目标大小"""
    n_original = len(X)
    if n_original >= target_size:
        return X, y

    X_expanded = []
    y_expanded = []


    augmentations_per_sample = target_size // n_original + 1

    for i in range(n_original):
        original_mat = X[i]
        original_label = y[i]

        # 為每個樣本產生多個增強版本
        augmented_versions = augment_matrix(original_mat)

        selected_augmentations = augmented_versions[:min(augmentations_per_sample, len(augmented_versions))]

        for aug_mat in selected_augmentations:
            X_expanded.append(aug_mat)
            y_expanded.append(original_label)

    while len(X_expanded) < target_size:
        idx = np.random.randint(0, n_original)
        X_expanded.append(X[idx])
        y_expanded.append(y[idx])

    X_expanded = np.array(X_expanded[:target_size])
    y_expanded = np.array(y_expanded[:target_size])


    # 打亂數據
    indices = np.random.permutation(len(X_expanded))
    X_expanded = X_expanded[indices]
    y_expanded = y_expanded[indices]

    return X_expanded, y_expanded


X_train_expanded, y_train_expanded = expand_dataset(X_train, y_train, target_size=10000)
print(f"扩展后数据集大小: {len(X_train_expanded)} 样本")

def data_generator(X, y, batch_size=32):
    n_samples = len(X)
    indices = np.arange(n_samples)
    np.random.shuffle(indices)

    while True:
        for start_idx in range(0, n_samples, batch_size):
            end_idx = min(start_idx + batch_size, n_samples)
            batch_indices = indices[start_idx:end_idx]

            batch_X = X[batch_indices]
            batch_y = y[batch_indices]

            yield batch_X, batch_y

# 4. 建構模型
model = Sequential([
    Flatten(input_shape=(8, 8)),
    Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# 5. 回呼函數
callbacks = [
    EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
]

# 6. 劃分訓練集和驗證集
split_idx = int(0.8 * len(X_train_expanded))
X_train_final = X_train_expanded[:split_idx]
y_train_final = y_train_expanded[:split_idx]
X_val = X_train_expanded[split_idx:]
y_val = y_train_expanded[split_idx:]

# 7. 訓練模型
batch_size = 32
steps_per_epoch = len(X_train_final) // batch_size

history = model.fit(
    data_generator(X_train_final, y_train_final, batch_size=batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=150,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

val_loss, val_accuracy, val_precision, val_recall = model.evaluate(X_val, y_val, verbose=0)
print(f"\n验证集结果:")
print(f"准确率: {val_accuracy:.4f}")
print(f"精确率: {val_precision:.4f}")
print(f"召回率: {val_recall:.4f}")

# 8. 修改後的預測函數 - 大於0.5就顯示為1
def predict_number(input_matrix):
    data = np.array(input_matrix).reshape(1, 8, 8).astype("float32")
    pred = model.predict(data, verbose=0)[0][0]

    if pred > 0.5:
        confidence = "（是1）"
        judgment = '1'
    else:
        confidence = "（不是1）"
        judgment = '3'

    print(f"预测概率: {pred:.4f}{confidence} -> 判断: {judgment}")
    return pred, judgment

sample_input1 = [
[0,0,0,0,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0]
]
#1
sample_input2 = [
[0,0,0,0,0,0,0,0],
[0,0,0,1,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,1,1,1,1,1,0],
[0,0,0,0,0,0,0,0]
]
#1
sample_input3 = [
[0,0,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0],
[0,1,1,0,0,0,0,0],
[0,0,1,0,0,0,0,0],
[0,0,1,0,0,0,0,0],
[0,1,1,1,1,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0]
]
#1
sample_input4 = [
[0,0,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0],
[0,1,1,0,0,0,0,0],
[0,0,1,0,0,0,0,0],
[0,0,1,1,0,0,0,0],
[0,0,1,1,1,1,1,0],
[1,1,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0]
]
#1
sample_input5 = [
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,0]
]

sample_input6 = [
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,1,1,1],
[0,0,0,0,1,1,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0]
]

sample_input7 = [
[0,0,0,0,0,0,0,0],
[0,0,1,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,1,1,1],
[0,1,1,1,1,0,0,0]
]
sample_input8 = [
[0,0,0,0,0,1,1,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,1,0,0],
[0,1,0,0,1,0,0,0],
[0,0,1,1,1,0,0,0],
[0,0,0,1,1,1,0,0],
[0,0,0,0,0,1,1,0]
]
sample_input9 = [
[0,0,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,0,0,0,0,0]
]
sample_input10 = [
[0,0,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0]
]

sample_input11 = [
[0,0,0,0,0,0,0,0],
[0,0,1,1,1,1,0,0],
[0,0,1,0,0,1,0,0],
[0,0,0,1,1,0,0,0],
[0,0,1,1,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,1,0,0,0],
[0,0,1,1,0,0,0,0]
]
#1
sample_input12 = [
[0,0,0,0,0,0,0,0],
[0,0,1,1,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,1,1,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,1,1,1,1,0,0,0],
[0,0,0,0,0,0,0,0]
]
#1
sample_input13 = [
[0,0,0,0,0,0,0,0],
[0,0,0,0,1,1,1,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,1,1,0],
[0,0,0,0,0,1,1,1],
[0,0,0,0,0,0,0,1],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,1,0,0]
]
#1
sample_input14 = [
[0,0,0,0,0,0,0,0],
[0,1,1,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,1,1,1,0,0,0,0],
[0,1,1,1,0,0,0,0],
[0,0,1,1,0,0,0,0],
[0,1,1,0,0,0,0,0],
[0,0,0,0,0,0,0,0]
]
#1
sample_input15 = [
[0,1,1,1,1,0,0,0],
[0,1,0,0,0,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,1,1,1,1,1,0],
[0,0,0,0,0,0,1,1],
[0,0,0,0,0,0,1,0],
[0,0,0,0,1,1,0,0],
[0,1,1,1,0,0,0,0]
]

sample_input16 = [
[0,0,0,0,1,0,0,0],
[0,0,0,0,0,1,0,0],
[0,1,1,1,0,1,0,0],
[1,0,0,1,1,0,1,0],
[1,1,0,0,1,1,0,0],
[0,1,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0],
[0,0,1,1,0,0,0,0]
]

sample_input17 = [
[0,0,0,0,0,0,0,0],
[0,1,1,1,1,0,0,0],
[0,1,0,0,1,1,0,0],
[0,0,0,0,1,1,0,0],
[0,0,1,1,1,0,0,0],
[0,0,0,0,0,1,0,0],
[0,1,0,0,1,0,0,0],
[0,0,1,1,1,0,0,0]
]
sample_input18 = [
[0,0,0,0,0,0,0,0],
[0,0,0,1,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,1,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,1,1,0,0,0],
[1,1,1,1,0,0,0,0]
]
sample_input19 = [
[0,0,0,0,0,0,0,0],
[0,0,0,1,1,0,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,1,1,0,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,1,1,0,0],
[1,1,1,1,0,0,0,0]
]
sample_input20 = [
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,1,1,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,1,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,1,0,0,0]
]

predict_number(sample_input1)
predict_number(sample_input2)
predict_number(sample_input3)
predict_number(sample_input4)
predict_number(sample_input5)
predict_number(sample_input6)
predict_number(sample_input7)
predict_number(sample_input8)
predict_number(sample_input9)
predict_number(sample_input10)
predict_number(sample_input11)
predict_number(sample_input12)
predict_number(sample_input13)
predict_number(sample_input14)
predict_number(sample_input15)
predict_number(sample_input16)
predict_number(sample_input17)
predict_number(sample_input18)
predict_number(sample_input19)
predict_number(sample_input20)
model.save('model.h5')

files.download('model.h5')

Saving x.npy to x (5).npy
扩展后数据集大小: 10000 样本
Epoch 1/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5380 - loss: 0.9328 - precision: 0.7007 - recall: 0.3849 - val_accuracy: 0.7005 - val_loss: 0.6825 - val_precision: 0.6820 - val_recall: 0.9320 - learning_rate: 5.0000e-04
Epoch 2/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6839 - loss: 0.7059 - precision: 0.7146 - recall: 0.7828 - val_accuracy: 0.7305 - val_loss: 0.6278 - val_precision: 0.7119 - val_recall: 0.9203 - learning_rate: 5.0000e-04
Epoch 3/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7200 - loss: 0.6532 - precision: 0.7393 - recall: 0.8198 - val_accuracy: 0.7620 - val_loss: 0.5848 - val_precision: 0.7393 - val_recall: 0.9279 - learning_rate: 5.0000e-04
Epoch 4/150
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7420 - loss: 0.6154 - precision: 0.7553 - recall: 0.8395 - val_accuracy: 0.7855 - val_loss: 0.5473 - val_precision: 0.7593 - val_recall: 0.9371 - learning_rate: 5.0000e-04

预测概率: 0.0001（不是1） -> 判断: 3
预测概率: 0.0003（不是1） -> 判断: 3


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>